In [4]:
cd /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---23-51
echo "pwd: $(pwd)"
DATE=$(date)
echo "date: $DATE"
MACHINE=$(uname -a)
echo "machine: $MACHINE"
echo "git status: $(git status --short)"
echo "git commit: $(git rev-parse HEAD)"
ROOT=$(git rev-parse --show-toplevel)
echo "root folder: $ROOT"
OUTFOLDER=$(pwd)
echo "out folder: $OUTFOLDER"

if [ "$(uname -s)" = "Darwin" ]; then
    TIME="gtime"
else
    TIME="command time"
fi
echo "time: $TIME"
$TIME -v echo "time"

pwd: /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---23-51
date: Fri 29 Mar 08:10:56 GMT 2024
machine: Linux dev-cpu-tcg40 6.5.0-26-generic #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2 x86_64 x86_64 x86_64 GNU/Linux
git status:  M run-bash.ipynb
git commit: 0415652bbab3dd84aecf77ab731a98ef4d83d8a7
root folder: /auto/homes/sb2743/2024-borrowing/lean4
out folder: /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-28---23-51
time: command time
time
	Command being timed: "echo time"
	User time (seconds): 0.00
	System time (seconds): 0.00
	Percent of CPU this job got: 100%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.00
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 1920
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 91
	Voluntary context switches: 1
	

## Benchmarking Strategy
- Bench on Linux.
- Bench stage2, stage3.

In [5]:
rm -rf $ROOT/build/noreuse || true
mkdir -p $ROOT/build/noreuse
cd $ROOT/build/noreuse

# == NO REUSE ==

echo "@@@ NOREUSE BUILD @@@"

CSVNAME=$(date +'%s---%d-%m-%Y---%H:%M:%S')
echo "CSV name is: $CSVNAME"
echo "output file is: OUTFOLDER/$CSVNAME"

cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=ON \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=0 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFOLDER/$CSVNAME

make -j stage1 2>1 >> log.txt
make -j stage2 2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-noreuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage2.csv
$TIME -v make -j stage3 &> "time-noreuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage3.csv

curl -d "Done[noreuse]\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2
# == REUSE ==

echo "@@@ REUSE BUILD @@@"

rm -rf $ROOT/build/reuse || true
mkdir -p $ROOT/build/reuse
cd $ROOT/build/reuse

CSVNAME="$(date +'%s---%d-%m-%Y---%H:%M:%S')"
echo "output file is: $OUTFOLDER/$CSVNAME"

cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=ON \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=1 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFOLDER/$CSVNAME
make -j stage1  2>1 >> log.txt
make -j stage2  2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-reuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage2.csv

$TIME -v make -j stage3 &> "time-reuse-stage3.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage3.csv

echo "DONE!"
curl -d "Done fully\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2

@@@ NOREUSE BUILD @@@
CSV name is: 1711699885---29-03-2024---08:11:25
output file is: OUTFOLDER/1711699885---29-03-2024---08:11:25
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /auto/homes/sb2743/2024-borrowing/lean4/build/noreuse
{"id":"pfIkkUJrh3R9","time":1711701845,"expires":1711745045,"event":"message","topic":"xISSztEV8EoOchM2","message":"Done[noreuse]\\n*machine:Linux dev-cpu-tcg40 6.5.0-26-generic #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:

In [7]:
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35
cp $ROOT/build/noreuse/time-noreuse-stage2.txt  $OUTFOLDER/time-noreuse-stage2.txt
cp $ROOT/build/noreuse/time-noreuse-stage3.txt  $OUTFOLDER/time-noreuse-stage3.txt

cp $ROOT/build/reuse/time-reuse-stage2.txt  $OUTFOLDER/time-reuse-stage2.txt  # TODO: fix this typo!
cp $ROOT/build/reuse/time-reuse-stage3.txt  $OUTFOLDER/time-reuse-stage3.txt